# Gibbs Sampling

By Alberto Valdés 

**Mail 1:** anvaldes@uc.cl 

**Mail 2:** alberto.valdes.gonzalez.96@gmail.com

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
import math
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import image as mpimg

In [3]:
def display_img(name, a, b):
    plt.figure(figsize = (a, b))
    image = mpimg.imread(name)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [4]:
start = time.time()

In statistics, Gibbs sampling or a Gibbs sampler is a Markov chain Monte Carlo (MCMC) algorithm for obtaining a sequence of observations which are approximated from a specified multivariate probability distribution, when direct sampling is difficult.

# Algorithm

$ \theta_0 $

for t in range (1, N):

$ \theta_t \sim p( \theta_t | \theta_{t-1} ) $

# Application to Statical Inference

We want to estimate parameters, and do inference about that.




### Example

We have two independent samples $ X_{1, 1}, ..., X_{1, n_1} $ and $ X_{2, 1}, ..., X_{2, n_2} $.

$ \mathbb{P} (X | \Theta) = \mathbb{P} (X_1 | \Theta) \cdot \mathbb{P} (X_2 | \Theta) \ \ \ $ with $ \ \ \ X_{i, j} | \Theta \sim N(\mu_i, \sigma) \ \ \ \forall i = 1, 2, j = 1, ..., n_i $

$ \ $

For example we have:

$ \ $

$ \mu_i | m, \tau^2 \sim N(m, \tau^2) $ for $ i = 1, 2 $

$ m | \sigma_m^2 \sim N(\mu_0, \sigma_m^2) $

$ \sigma^2 \sim GI(a_1, b_1) $

$ \tau^2 \sim GI(a_2, b_2) $

$ \sigma^2_m \sim GI(a_3, b_3) $

$ \ $

If we define $ n = n_1 + n_2, \bar{u} = \mu_1, \mu_2 $ and $ \bar{x}_i = \cfrac{1}{n_i} \sum_{j=1}^{n_i} X_{i, j} $ and using that $ \mathbb{P} (\Theta | X) \propto \mathbb{P} (X | \Theta) \cdot \mathbb{P} (\Theta)  $ we can prove that:

$ \ $

$ (\mu_i | \text{Everything else}) \sim N \left( \cfrac{\sigma^2 \cdot m + n_i \cdot \tau^2 \cdot \bar{x}_i}{\sigma^2 + n_i \cdot \tau^2}, \cfrac{\sigma^2 \cdot \tau^2}{\sigma^2 + n_i \cdot \tau^2} \right) $ for $ i = 1, 2 $

$ (m | \text{Everything else}) \sim N \left( \cfrac{\tau^2 \cdot \mu_0 + 2 \cdot \sigma_m^2 \cdot \bar{u}}{\tau^2 + 2 \cdot \sigma_m^2}, \cfrac{\sigma_m^2 \cdot \tau^2}{\tau^2 + 2 \cdot \sigma_m^2} \right) $

$ (\sigma^2 | \text{Everything else}) \sim GI \left( a_1 + \cfrac{n}{2}, b_1 + \sum_{i=1}^2 \sum_{j=1}^{n_i} (X_{i, j} - \mu_i)^2 \right) $

$ (\tau^2 | \text{Everything else}) \sim GI \left( a_2 + 1, b_2 + \cfrac{1}{2} \cdot \sum_{i=1}^2 (\mu_{i} - m)^2 \right) $

$ (\sigma_m^2 | \text{Everything else}) \sim GI \left( a_3 + \cfrac{1}{2}, b_3 + \cfrac{1}{2} \cdot (m - \mu_0)^2 \right) $

In [5]:
X_1 = [81.1, 80.2, 83.7, 74.7, 79.5, 78.6, 78.5, 86.5, 74.3, 82.6, 86.8, 87.6, 71.0, 72.5, 73.8, 81.9, 72.3, 77.3, 82.2, 68.8, 83.3, 80.6, 73.0, 77.9, 81.9]
X_2 = [83, 86.7, 58.1, 69.2, 85.5, 72.7, 75.8, 65.8, 79.8, 66.4, 58.0, 66.4, 64.1, 68.2, 58.2, 62.5, 64.7, 75.3]

X_1 = pd.Series(X_1)
X_2 = pd.Series(X_2)

In [6]:
bar_X_1 = X_1.mean()
bar_X_2 = X_2.mean()

n_1 = len(X_1)
n_2 = len(X_2)

n = n_1 + n_2

a1=3
b1=3

a2=3
b2=3

a3=3
b3=3

In [7]:
mu_1_0 = 75
mu_2_0 = 75

mu_0 = 75

sigma_2_0 = 1
tau_2_0 = 1
sigma_m_2_0 = 1

m_0 = 75

In [8]:
mu_1 = []
mu_2 = []

sigma_2 = []
tau_2 = []
sigma_m_2 = []

m = []

In [9]:
mu_1.append(mu_1_0)
mu_2.append(mu_2_0)

sigma_2.append(sigma_2_0)
tau_2.append(tau_2_0)
sigma_m_2.append(sigma_m_2_0)

m.append(m_0)

In [10]:
mu_1_now = mu_1_0
mu_2_now = mu_2_0

sigma_2_now = sigma_2_0
tau_2_now = tau_2_0
sigma_m_2_now = sigma_m_2_0

m_now = m_0

In [11]:
T = 1_000_000

In [12]:
for t in range(T):

    #----------------------------------------------------------------------

    mean_mu_1 = (sigma_2_now*m_now + n_1*tau_2_now*bar_X_1)/(sigma_2_now + n_1*tau_2_now)
    var_mu_1 = (sigma_2_now*tau_2_now)/(sigma_2_now + n_1*tau_2_now)

    mean_mu_2 = (sigma_2_now*m_now + n_2*tau_2_now*bar_X_2)/(sigma_2_now + n_2*tau_2_now)
    var_mu_2 = (sigma_2_now*tau_2_now)/(sigma_2_now + n_2*tau_2_now)
    
    #----------------------------------------------------------------------

    u_prom_now = (mu_1_now + mu_2_now)/2

    #----------------------------------------------------------------------

    mean_m = (tau_2_now*mu_0 + 2*sigma_m_2_now*u_prom_now)/(tau_2_now + 2*sigma_m_2_now)
    var_m = (sigma_m_2_now*tau_2_now)/(tau_2_now + 2*sigma_m_2_now)

    #----------------------------------------------------------------------

    sigma_2_param_1 = a1 + (n/2)
    sigma_2_param_2 = b1 + ((X_1 - mu_1_now)**2).sum() + ((X_2 - mu_2_now)**2).sum()

    #----------------------------------------------------------------------

    tau_2_param_1 = a2 + 1
    tau_2_param_2 = b2 + (1/2)*((mu_1_now - m_now)**2 + (mu_2_now - m_now)**2)

    #----------------------------------------------------------------------

    sigma_m_2_param_1 = a3 + (1/2)
    sigma_m_2_param_2 = b3 + (1/2)*((m_now - mu_0)**2)

    #----------------------------------------------------------------------

    #Updating variables 

    mu_1_now = np.random.normal(mean_mu_1, var_mu_1**(1/2), 1)[0]
    mu_2_now = np.random.normal(mean_mu_2, var_mu_2**(1/2), 1)[0]

    m_now = np.random.normal(mean_m, var_m**(1/2), 1)[0]
    
    sigma_2_now = 1/np.random.gamma(sigma_2_param_1, sigma_2_param_2, 1)[0]
    tau_2_now = 1/np.random.gamma(tau_2_param_1, tau_2_param_2, 1)[0]
    sigma_m_2_now = 1/np.random.gamma(sigma_m_2_param_1, sigma_m_2_param_2, 1)[0]

    #----------------------------------------------------------------------

    #Adding values

    mu_1.append(mu_1_now)
    mu_2.append(mu_2_now)
    
    m.append(m_now)

    sigma_2.append(sigma_2_now)
    tau_2.append(tau_2_now)
    sigma_m_2.append(sigma_m_2_now)

    #----------------------------------------------------------------------

In [13]:
mu_1 = pd.Series(mu_1)
mu_2 = pd.Series(mu_2)

m = pd.Series(m)

sigma_2 = pd.Series(sigma_2)
tau_2 = pd.Series(tau_2)
sigma_m_2 = pd.Series(sigma_m_2)

In [14]:
round(mu_1[-10:].mean(), 2)

78.82

In [15]:
round(mu_2[-10:].mean(), 2)

70.02

**Comments:** As you can see $ \mu_1 > \mu_2 $.

### Time of execution

In [16]:
end = time.time()

In [17]:
delta = (end - start)

hours = int(delta/3600)
mins = int((delta - hours*3600)/60)
segs = int(delta - hours*3600 - mins*60)
print(f'Execute this notebook take us {hours} hours, {mins} minutes and {segs} seconds.')

Execute this notebook take us 0 hours, 2 minutes and 51 seconds.
